In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
kern2017 = pd.read_csv("data/groningen/groningen_kern2017.csv", sep=';')

In [ ]:
def bla(columname):
    splitted = columname.split('_')
    if len(splitted) > 1:
        splitted = splitted[:-1]
        if len(splitted) > 1:
            return '_'.join(splitted)
    return splitted[0]

In [ ]:
columns = kern2017.columns.map(bla)

In [ ]:
import re
df = pd.read_csv('../data/alle_jaren_zonder_de_kut_puntjes.csv')
df = df.drop("WijkenEnBuurten", axis=1).drop("SoortRegio", axis=1).drop(df.columns[0], axis=1)
# for x in df.columns:
#     if type(df[x][0]) == str:
#         df[x] = df[x].str.replace(' ', '')

## Prediction pipeline

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
rmses = []
rmses_baseline = []
for i in range(50):
    msk = np.random.rand(len(df)) < 0.9
    train_set = df[msk]
    test_set = df[~msk]

    y_train = train_set['attributes.AANTAL_SUMMED'].values
    X_train = train_set.drop('attributes.AANTAL_SUMMED', axis=1).drop('attributes.AANTAL',axis=1).values

    y_test = test_set['attributes.AANTAL_SUMMED'].values
    X_test = test_set.drop('attributes.AANTAL_SUMMED', axis=1).drop('attributes.AANTAL',axis=1).values

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2', 'rmse'},
        'num_leaves': 127,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=10)


    print('Start predicting...')
    # predict
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    print(y_pred)
    print(y_test)
    baseline = [np.mean(y_train)] * len(y_test)
    # eval
    print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
    print('The rmse of baseline is:', mean_squared_error(baseline, y_test) ** 0.5)
    rmses.append(mean_squared_error(y_test, y_pred) ** 0.5)
    rmses_baseline.append(mean_squared_error(baseline, y_test) ** 0.5)
print(np.mean(rmses))
print(np.mean(rmses_baseline))


In [ ]:
msk = np.random.rand(len(df)) < 0.9
train_set = df[msk]
test_set = df[~msk]

y_train = train_set['attributes.AANTAL_SUMMED'].values
X_train = train_set.drop('attributes.AANTAL_SUMMED', axis=1).drop('attributes.AANTAL',axis=1).values
# X_train = X_train[['HuishoudensZonderKinderen','k_45Tot65Jaar','k_65JaarOfOuder','Gehuwd','Gescheiden','BouwjaarVoor2000','TweeOnderEenKapWoning1',
# 'GemiddeldAardgasverbruikTotaal','AantalInkomensontvangers','GemiddeldInkomenPerInkomensontvanger','HuishoudensMetEenLaagInkomen',
# 'HuishOnderOfRondSociaalMinimum','BedrijfsvestigingenTotaal','PersonenautoSOverigeBrandstof'].values

y_test = test_set['attributes.AANTAL_SUMMED'].values
X_test = test_set.drop('attributes.AANTAL_SUMMED', axis=1).drop('attributes.AANTAL',axis=1).values
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'rmse'},
    'num_leaves': 127,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
gbm.save_model('../data/model.txt')